In [15]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
from config import apikey, email
from g_key import gkey
import gmaps
import os

In [16]:
# base url for state codes needed down the line to focus in 
state_url = (f'https://aqs.epa.gov/data/api/list/states?email={email}&key={apikey}')

# get the state codes into a data frame
response=requests.get(state_url)
data=response.json()
data

{'Header': [{'status': 'Success',
   'request_time': '2021-02-04T00:19:48-05:00',
   'url': 'https://aqs.epa.gov/data/api/list/states?email=erich.mitchell87@gmail.com&key=goldmouse56',
   'rows': 56}],
 'Data': [{'code': '01', 'value_represented': 'Alabama'},
  {'code': '02', 'value_represented': 'Alaska'},
  {'code': '04', 'value_represented': 'Arizona'},
  {'code': '05', 'value_represented': 'Arkansas'},
  {'code': '06', 'value_represented': 'California'},
  {'code': '08', 'value_represented': 'Colorado'},
  {'code': '09', 'value_represented': 'Connecticut'},
  {'code': '10', 'value_represented': 'Delaware'},
  {'code': '11', 'value_represented': 'District Of Columbia'},
  {'code': '12', 'value_represented': 'Florida'},
  {'code': '13', 'value_represented': 'Georgia'},
  {'code': '15', 'value_represented': 'Hawaii'},
  {'code': '16', 'value_represented': 'Idaho'},
  {'code': '17', 'value_represented': 'Illinois'},
  {'code': '18', 'value_represented': 'Indiana'},
  {'code': '19', 'va

In [3]:
# parameter classes
param_url=f'https://aqs.epa.gov/data/api/list/parametersByClass?email={email}&key={apikey}&pc=CRITERIA'

response=requests.get(param_url)
data_param=response.json()
data_param

{'Header': [{'status': 'Success',
   'request_time': '2021-02-04T00:06:28-05:00',
   'url': 'https://aqs.epa.gov/data/api/list/parametersByClass?email=erich.mitchell87@gmail.com&key=goldmouse56&pc=CRITERIA',
   'rows': 8}],
 'Data': [{'code': '14129', 'value_represented': 'Lead (TSP) LC'},
  {'code': '42101', 'value_represented': 'Carbon monoxide'},
  {'code': '42401', 'value_represented': 'Sulfur dioxide'},
  {'code': '42602', 'value_represented': 'Nitrogen dioxide (NO2)'},
  {'code': '44201', 'value_represented': 'Ozone'},
  {'code': '81102', 'value_represented': 'PM10 Total 0-10um STP'},
  {'code': '85129', 'value_represented': 'Lead PM10 LC FRM/FEM'},
  {'code': '88101', 'value_represented': 'PM2.5 - Local Conditions'}]}

In [4]:
# pulling sample data
# Choose state
state='53'

# choose parameter 
param='42101'

# create the begin date and end date for search using yyyymmdd format
bdate='20200131'
edate='20201231'

# develop the query URL - Sample Data
sample_url=f'https://aqs.epa.gov/data/api/sampleData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state={state}'
response=requests.get(sample_url)
sample_param=response.json()
sample_param['Data'][0].keys()


dict_keys(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'date_local', 'time_local', 'date_gmt', 'time_gmt', 'sample_measurement', 'units_of_measure', 'units_of_measure_code', 'sample_duration', 'sample_duration_code', 'sample_frequency', 'detection_limit', 'uncertainty', 'qualifier', 'method_type', 'method', 'method_code', 'state', 'county', 'date_of_last_change', 'cbsa_code'])

In [5]:
#annual summary data - pulling same params from previous kernel - Annual Data by state
annual_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state={state}'
response=requests.get(annual_url)
annual_param=response.json()
annual_param['Data'][0].keys()


dict_keys(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'sample_duration', 'pollutant_standard', 'metric_used', 'method', 'year', 'units_of_measure', 'event_type', 'observation_count', 'observation_percent', 'validity_indicator', 'valid_day_count', 'required_day_count', 'exceptional_data_count', 'null_observation_count', 'primary_exceedance_count', 'secondary_exceedance_count', 'certification_indicator', 'arithmetic_mean', 'standard_deviation', 'first_max_value', 'first_max_datetime', 'second_max_value', 'second_max_datetime', 'third_max_value', 'third_max_datetime', 'fourth_max_value', 'fourth_max_datetime', 'first_max_nonoverlap_value', 'first_max_n_o_datetime', 'second_max_nonoverlap_value', 'second_max_n_o_datetime', 'ninety_ninth_percentile', 'ninety_eighth_percentile', 'ninety_fifth_percentile', 'ninetieth_percentile', 'seventy_fifth_percentile', 'fiftieth_percentile', 'tenth_percentile', 'local_site_name', 's

In [6]:
# hard coded searches
param='42101'
bdate='20200131'
edate='20201231'

#query URL
query_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state='

# response = requests.get(query_url + code).json()


In [8]:
# Set up empty lists for data in each column
states = []
lat = []
long = []
mean = []
max_value = []
# Create a list of state codes so our request can loop through and pull each state
state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]
# Loop through each state code and pull mean and max No2 values for all locations in each state
for code in state_code:
    response = requests.get(query_url + code).json()
    try:
        states.extend([x['state'] for x in response['Data']])
        lat.extend([x['latitude'] for x in response['Data']])
        long.extend([x['longitude'] for x in response['Data']])
        mean.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value.extend([x['first_max_value'] for x in response['Data']])
        
    except: 
        print("This state has no data", code)
        

In [12]:
states_df=pd.DataFrame({'State':states,
           'Latitude': lat,
           'Longitude':long,
           'Mean Value': mean,
           'Max Value': max_value})

states_df

,State,Latitude,Longitude,Mean Value,Max Value
0,Alabama,33.553056,-86.815000,0.232099,1.900
1,Alabama,33.553056,-86.815000,0.233456,1.200
2,Alabama,33.521427,-86.844112,0.333946,3.164
3,Alabama,33.521427,-86.844112,0.336042,1.300
4,Alabama,33.485556,-86.915000,0.183963,1.200
...,...,...,...,...,...
546,Puerto Rico,18.422595,-66.120012,0.528008,0.800
547,Puerto Rico,18.009558,-66.627249,0.165382,22.800
548,Puerto Rico,18.009558,-66.627249,0.164927,3.100
549,Puerto Rico,18.449814,-66.052510,0.967785,5.700


In [14]:
states_df_grouped=states_df.groupby('State').first()
states_df_grouped

,Latitude,Longitude,Mean Value,Max Value
State,,,,
Alabama,33.553056,-86.815000,0.232099,1.900
Alaska,61.205861,-149.824602,0.385518,4.372
Arizona,33.483780,-112.142560,0.505573,3.800
Arkansas,34.756189,-92.281296,0.286346,1.300
California,33.830620,-117.938450,0.339064,2.100
Colorado,38.830895,-104.839243,0.324609,1.853
Connecticut,41.821342,-73.297257,0.148332,4.142
Delaware,39.739444,-75.558056,0.223580,1.000
District Of Columbia,38.921847,-77.013178,0.238271,2.007


In [25]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = states_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = states_df["Max Value"].astype(float)

In [26]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlong, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
file_path="Net_generation_for_all_sectors (1).csv"
df = pd.read_csv(file_path)
df

,Net generation for all sectors,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241
0,https://www.eia.gov/electricity/data/browser/#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Jan 30 2021 12:20:00 GMT-0700 (Mountain St...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: U.S. Energy Information Administration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,description,units,source key,Jan-01,Feb-01,Mar-01,Apr-01,May-01,Jun-01,Jul-01,...,Feb-20,Mar-20,Apr-20,May-20,Jun-20,Jul-20,Aug-20,Sep-20,Oct-20,Nov-20
4,Net generation for all sectors,thousand megawatthours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,Hawaii : hydro-electric pumped storage,thousand megawatthours,ELEC.GEN.HPS-HI-99.M,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1303,Hawaii : other,thousand megawatthours,ELEC.GEN.OTH-HI-99.M,12,12,5,15,17,19,18,...,25,30,23,27,21,30,28,21,19,31
1304,Hawaii : all solar,thousand megawatthours,ELEC.GEN.TSN-HI-99.M,--,--,--,--,--,--,--,...,119,136,153,167,164,168,167,155,140,123
1305,Hawaii : small-scale solar photovoltaic,thousand megawatthours,ELEC.GEN.DPV-HI-99.M,--,--,--,--,--,--,--,...,87,99,107,114,113,116,115,106,99,87
